<br><br><br><br>


## Emperor Penguin Study - Data Preparation

Data for the penguin study is stored in a directory structure on the file system. Penguin colonies are the highest directory, then classification studies by date and image catalog id directories, which contain the study shapefiles and signature file.

This notebook navigates the project directories and creates useful data structures and image tiles.


<br><br><br>
## SECTION: Initialize


#### Naming conventions

##### colony - the name of a penguin colony as used in the project directory structure. Colony names do not have spaces and the individual parts are capitalized. ie 'Amanda Bay' becomes 'AmandaBay'

##### prjShapeCatalog (project shapefile catalog) - a list of all of the shapefiles found in the project directory structure. Only the file with the '.shp' suffix is in this list. Each entry contains the entire path to the file.

##### imgCatalog (images catalog) - a list containing the entire path to every image file (.tif) on the mounted external drives.

##### prjCatalog (projects catalog) - a list of individual project directories found in the project directory structure. These directories are named by concatenating colony name, year, month day (optional), and catalog id(s) separated by "_". ie 'KloaPoint_2013_1010010011FFD500'

##### Projects data structure - a structured list of [colony name, year, month day, [catalog ids], project directory name]
 
 








#### If needed, install libraries:

In [ ]:
!pip install rasterio geopandas fiona

#### Import libraries

In [2]:
#
# Import needed libraries
#

import rasterio as rio
import glob
import os

import geopandas as gpd
import fiona

# Enable fiona driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


#### Constants and search paths

In [3]:
#
# Constants
#

# Different tile sizes, used for buffering...
NONE = 0
TILE16 = 1
TILE32 = 2


In [4]:
#
# Set the search paths...
#

mount_dir = "/media"
proj_home = "/home/tomg/projects/penguins/data/colonies"


<br><br><br>
## SECTION: Functions to find and organize data 


In [5]:
#
# Get a catalog of colony names...
#

def coloniesCatalog(proj_home=proj_home):
        
    # Find ALL of the colony names...
    cols = [ name for name in os.listdir(proj_home) if os.path.isdir(os.path.join(proj_home, name)) ]

    return cols


In [6]:
#
# Get a catalog of shapefiles...
#

def prjShpCatalog(proj_home=proj_home,name=''):

    # Find all of the PROJECT shapefiles in "proj_home"...
    files = glob.glob(proj_home+"/**/*.shp",recursive=True)
    shps = [shp for shp in files if name.upper() in shp.upper()]
    
    return shps


In [7]:
#
# Get a catalog of ALL satellite images...
#   This finds all tif images on the mounted external drives.

def imgCatalog(mount_dir=mount_dir):
        
    # Find ALL of the tif files...
    imgs = glob.glob(mount_dir+"/**/*.tif",recursive=True)

    return imgs


In [8]:
#
# Get a catalog of projects (colony and year combinations)...
#

def prjCatalog(proj_home=proj_home):
    
    prjs = []

    dirs = glob.glob(proj_home+"/**/*",recursive=False)
    prjs = [os.path.basename(p) for p in dirs]
    return prjs


In [9]:
#
# Parse the project directory name, return a list
#   [colony, year, month day, [catalog id list]]
#

def parse_project(prj):
    
    months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
    month = ''
    hasMonth = False
    firstCatId = 2
    
    catid_list = []
    
    p_list = prj.split('_')
    
    # TO-DO Probably should check that the length of p_list is at least 3...
    if not (len(p_list) >= 3):      #Prj needs at least colony, year, and catalog id.
        return None

    colony, year, unk = p_list[:3]
    
    # Some projects include month and day...
    for m in months:
        if m in unk:
            # Found a month, so set it here
            month = unk
            hasMonth = True

    # If project has a month, catalog ids start with index=3, else index=2...
    if hasMonth:
        firstCatId = 3

    # Append catalog ids starting at the 4th positon...
    
    for c in p_list[firstCatId:]:
        catid_list.append(c)
    
    return [colony, year, month, catid_list]



In [10]:
#
# Given a search path, function returns a list of projects
#   as a list: colony, year, catalog id, project directory

def get_projects():
    
    projects = []
    
    for p in prjCatalog():
        if not '.' in p:
            projects.append([parse_project(p),p]) 

    return projects
    
    

In [11]:
#
# Get the bounds of the entire shapefile and buffer it by "buffer"...
#   total_bounds returns coordinates in the image's coordinate system
#   in this order: left, bottom, right, top
#

def get_bounds(shp, buffer=NONE):

    b = gpd.read_file(shp).total_bounds

    if buffer == NONE:
        return b
    
    elif buffer == TILE16:  # Specific for 16x16 training thumbnails...
        b[0] -= 18
        b[1] -= 18
        b[2] += 26
        b[3] += 26
        return b
    
    elif buffer == TILE32:  # Specific for 32x32 training thumbnails...
        b[0] -= 36
        b[1] -= 36
        b[2] += 56
        b[3] += 56
        return b
    
    else:
        b[0] -= buffer
        b[1] -= buffer
        b[2] += buffer
        b[3] += buffer
        return b

            

In [12]:
#
# create a filename with path for the clipped file
#

def clipped_outfile(prj_name,path_name,img):

    colony, year, month, (catalog_id) = prj_name
    path = path_name
    
    cat_id = [c_id for c_id in catalog_id if c_id in img][0]

    if month > '':
        year += year + '_' + month
        
#    colony,yr,cat_id,path = project

    return path + '/' + colony.lower() + year + '_' + cat_id + 'clipped.tif'
    
    

In [13]:
#
#  This cell does the heavy lifting of clipping the original image and saving as a new image file
#
#   Inspiration: 
#     https://gis.stackexchange.com/questions/367832/using-rasterio-to-crop-image-using-pixel-coordinates-instead-of-geographic-coord
#   Using "from_bounds" and "windows":
#     https://rasterio.groups.io/g/main/topic/window_from_bounds_behaviour/69541972?p=,,,20,0,0,0::recentpostdate%2Fsticky,,,20,2,0,69541972
#

def clip_image(dst_file, img, shp, bounds):

    left,bottom,right,top = bounds
    
    with rio.open(img) as src:

        # Check that bounds all fit within this image...
        if (src.bounds.left < left and src.bounds.bottom < bottom and
            src.bounds.right > right and src.bounds.top > top):

            # Create a window using from_bounds, with a 200m buffer...
            window = rio.windows.from_bounds(left,bottom,right,top,src.transform)
            # Use the transform of the source image...
            transform = src.window_transform(window)

            # Start with the profile from the source image...
            profile = src.profile
            # Update the profile of the window with the new height, width...
            profile.update({
                'height':    int(window.height),
                'width':     int(window.width),
                'transform': transform })

            # Read the source using the new window, and write the output file...
            print('Writing file {}'.format(dst_file))
            print('Bounds - {}'.format(bounds))
            with rio.open(dst_file,'w',**profile) as dst:
                dst.write(src.read(window=window))
            return True
        
        print('Bounding box of shapefile {} not in \nimage file: {}.\n'.format(shp,img))
        return False
    
    for c_id in catalog_id:
        if c_id in img:
            cat_id = c

In [14]:
#
# Match projects to existing shapefiles and images on catalog id...
#

def create_prj_list(projects, shps, imgs):

    prj_list = []

    for p in projects:
        colony, year, month, (catalog_id) = p[0]
        prj_path = p[1]
        
        # Look for path names that match the catalog id and contain shapefiles...
        shp_list = []
        for s in shps:
            if prj_path in s:
                shp_list.append(s)

        # Now look for images matching the catalog id...
        img_list = []
        name_list = []

        for cat_id in catalog_id:
            for i in imgs:
                if ((cat_id in i) and (".tif" in i) and not ("clipped" in i)) \
                        and (os.path.basename(i) not in name_list):
                    img_list.append(i)
                    name_list.append(os.path.basename(i))

        prj_list.append([p, shp_list, img_list])

    return prj_list


<br><br><br>
## SECTION: Create the data structures

This section reads project data and creates the required data structures.


### Find projects, shapefiles and images.

In [15]:
%%time

#
# Create the required lists
#

# Get the projects, shapefile list and image list...
projects = get_projects()
shps_cat = prjShpCatalog()
imgs_cat = imgCatalog()

print('Found shapefiles - {}, images - {}'.format(len(shps_cat), len(imgs_cat)))


Found shapefiles - 561, images - 18237
CPU times: user 610 ms, sys: 542 ms, total: 1.15 s
Wall time: 2.89 s


<br>

### Create a project list

In [16]:
#
# Combine the lists by colony, project, shapefile, images
#

# Create a project list data structure from the lists...
prj_list = create_prj_list(projects, shps_cat, imgs_cat)

print('Found {} projects'.format(len(prj_list)))


Found 222 projects


<br><br><br>
## SECTION - Create the needed data objects


### Create clipped images
For each colony and study season, clip the satellite image used for classification to the bounding box of identified penguins. Various buffers can be applied depending on the purpose (tile size, for example).

In [ ]:
%%time
#
# Create clipped images from original satellite imagery. Use classification shapefile and buffer.
#   (This takes a while...)
#

# Loop through the project list
for p in prj_list:

    prj,shps,imgs = p
    prj_name, path_name = prj
    colony,yr,month,(cat_id) = prj_name

    print("\nProcessing {} {} {}".format(colony,yr,cat_id))
    
    # Look for classification shapefile...
    for s in shps:
        if "CLASS" in os.path.basename(s.upper()):
            print("found {}\n".format(s))
            # Get the directory path to use for the clipped file...
            path = os.path.dirname(s)
            
            # Get the bounding box of the shapefile (left,bottom,right,top...)
            
            #   Set the type of bounds buffer:
            #     One of NONE, TILE16, TILE32, or an integer > 2. The TILE16 and TILE32 types are
            #       asymetrical buffer expecially training images.
            b = get_bounds(s,TILE32)

            # Process list of matching images...
            for i in imgs:
                # Progress report...
                print("checking image {}\n".format(i))
                # Create a output file name...
                #   Put clipped image file in project directory...
                outfile = clipped_outfile(prj_name,path,i)

                # Actually clip and save the file...
                if clip_image(outfile,i,s,b):  # outfile name, image path, shapefile, bounds
                    break                      # found the right image, stop looking



<br>

### Create colony bounding box shapefile
Using the bounding box of all  the clipped images, create a bounding box for each penguin colony. Write all colony bounding boxes to a single shapefile. 

##### ***Requires study bounding boxes, above.


In [18]:
#
# Find the bounding box of all of the clipped images for a colony...
#   Match colonies to existing clipped images...
#

# Find all of the colonies "proj_home"...
colonies = coloniesCatalog()
bnds = []

for colony in colonies:

    # Find all of the clipped images for current colony...
    path = proj_home+"/"+colony+"/**/*clipped.tif"
    files = glob.glob(path,recursive=True)
    
    # Check that there is at least one clipped image file...
    if len(files) > 0:
        first = True
        for file in files:
            with rio.open(file) as src:

                if first:
                    left = src.bounds.left
                    bottom = src.bounds.bottom
                    right = src.bounds.right
                    top = src.bounds.top

                    first = False

                else:
                    left = min(left,src.bounds.left)
                    bottom = min(bottom,src.bounds.bottom)
                    right = max(right,src.bounds.right)
                    top = max(top,src.bounds.top)

                    
        # Buffer bounds...
        buffer = 0.

        left -= buffer
        bottom -= buffer
        right += buffer
        top += buffer
                    
        geom = 'POLYGON(('+str(left)+' '+str(top)+', '+str(right)+' '+str(top)+', ' \
            +str(right)+' '+str(bottom)+', '+str(left)+' '+str(bottom)+', '+str(left)+' '+str(top)+'))'            

        bnds.append([colony,geom])

    # Convert list to geodataframe...
    gdf = gpd.GeoDataFrame(bnds)
    gdf.columns=['colony','wkt']
    gdf['geometry']=gpd.GeoSeries.from_wkt(gdf['wkt'])

    # Drop the 'wkt' column...
    gdf.drop(columns='wkt',inplace=True)

    # Write shapefile...
    gdf.to_file(proj_home+"/colony_boundaries.shp",crs="EPSG:3031")


<br>

### Create colony bounding box in KML format
Using the bounding box of all  the clipped images, create a single bounding box KML file for the penguin colony.
##### ***Requires study bounding boxes, above.

In [19]:
#
# Create a kml file for each colony...
#

for index,row in gdf.iterrows():
    shp_path = proj_home+"/"+row['colony']+"/"+row['colony']+"_extent.kml"

    gdf[index:index+1].to_file(shp_path,driver="KML",crs="EPSG:3031")



<br>

### Create classified tiles
Create penguin and no_penguin tiles from the clipped images. 
##### ***Requires clipped images, above.

Note: This takes about 5 hours to run on an old desktop

In [ ]:
%%time
#
# Tile individual images from each project clipped image...
#

from shapely.geometry import box, polygon
from rasterio.windows import Window

# Set the tile size for this step...
tile_size = 32

# Loop through the project list
for p in prj_list:

    prj,shps,imgs = p
    prj_name, path_name = prj
    colony,yr,month,(cat_id) = prj_name
    
    # Look for classification shapefile. Use either the classified point shapefile or the 
    #   penguin polygon shapefile...
    for s in shps:
        if os.path.basename(s.upper()).startswith("POLY"):

#    Penguins shapefile contains all four classifications. The PolyPen 
#      shapefile contains only penguins. For now, use PolyPen
#        if os.path.basename(s.upper()).startswith("PEN") or \
#              os.path.basename(s.upper()).startswith("POLY"):

            print("found shapefile - {}".format(s))

            # Look for the corresponding clipped tif file...
            #   Just look for a clipped.tif file in the directory we just found...
            path = os.path.dirname(s)
            file = glob.glob(path+"/*clipped.tif")  # There is only one!
            if file:
                print("found img file - {}, tiling...\n".format(file[0]))

                # Read the penguins polygons into a gdf...
                gdf = gpd.read_file(s)
                penguins = gdf.loc[gdf.gridcode==1]
                
                # Read the clipped file
                with rio.open(file[0]) as src:
                    
                    cols = src.width
                    rows = src.height
                    
                    print(cols,rows)
                    for r in range(0,cols-tile_size,tile_size):
                        for c in range(0,rows-tile_size,tile_size):
                    
                            # Create a window using offsets and size...
                            window = Window(r,c,width=tile_size,height=tile_size)
                            
                            transform = src.window_transform(window)
                            profile = src.profile
                            profile.update({
                                'height': window.height,
                                'width': window.width,
                                'transform': transform })
                            
                            with rio.open('/tmp/tmp_file.tif','w',**profile) as dst:

                                tile_bounds = dst.bounds
                                geom = box(*tile_bounds)
                                dst.close()
                               
                                if penguins.intersects(geom).any():
                                    dst_file = '/home/tomg/projects/penguins/models/train/penguins/'+path_name+'_{:04d}'.format(r)+'{:04d}'.format(c)+'.tif'
                                    with rio.open(dst_file,'w',**profile) as dst:
                                        dst.write(src.read(window=window))
                                        dst.close()
                                    
                                else:
                                    dst_file = '/home/tomg/projects/penguins/models/train/nopenguins/'+path_name+'_{:04d}'.format(r)+'{:04d}'.format(c)+'.tif'
                                    with rio.open(dst_file,'w',**profile) as dst:
                                        dst.write(src.read(window=window))
                                        dst.close()

                                    

<br><br><br>